In [12]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'boto3'

In [7]:
# Adapter Layer

def read_csv_to_df(bucket, objects):
    # Se obtiene el objeto inicial por posición en el bucket objects
    csv_obj_init = bucket.Object(key=objects[0].key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj_init)
    df_init = pd.read_csv(data, delimiter=',')

    # Concatenar todos los objetos con pandas, como son más, tarda más
    df_all = pd.DataFrame(columns=df_init.columns)
    for obj in objects:
        csv_obj = bucket.Object(key=obj.key).get().get('Body').read().decode('utf-8')
        data = StringIO(csv_obj)
        df = pd.read_csv(data, delimiter=',')
        df_all = pd.concat([df,df_all], ignore_index=True)

    return df_all

def write_df_to_s3(df_all, key):
    out_buffer = BytesIO()
    df_all.to_parquet(out_buffer, index=False)
    bucket_target = s3.Bucket('xetra-ajlj')
    bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)
    pass

def return_objects(bucket, arg_date_dt):
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split("/")[0], '%Y-%m-%d').date() >= arg_date_dt]
    return objects

In [8]:
# Application Layer

def extract():
    read_csv_to_df()
    return_objects()
    pass

def transform_report():
    pass

def load():
    key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
    pass

def etl_report():
    trg_bucket = 'xetra-bucket-12345'
    s3 = boto3.resource('s3')
    bucket_trg = s3.Bucket(trg_bucket)
    for obj in bucket_trg.objects.all():
        print(obj.key)
    pass
    

In [9]:
# main function entrypoint

def main():
    # Parameters/Configurations    
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('xetra-1234')

    # Se establece el argumento
    arg_date = '2022-12-30'


    # strptime: https://www.geeksforgeeks.org/python-datetime-strptime-function/
    arg_date_dt = datetime.strptime(arg_date, '%Y-%m-%d').date() - timedelta(days=1)

    # Init    
    
    
    # run application
    

In [10]:
# run

main()

NameError: name 'boto3' is not defined

## Reading the uploaded file

In [4]:
trg_bucket = 'xetra-bucket-12345'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20220215_115050.parquet
xetra_daily_report_20220218_134804.parquet
xetra_daily_report_20220308_103748.parquet
xetra_daily_report_20220310_110626.parquet


In [5]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20220310_110626.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [6]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-03-10,35.85,35.85,35.85,35.85,0,-1.65
1,AT00000FACC2,2022-03-10,7.41,7.35,7.35,7.41,200,-0.68
2,AT0000606306,2022-03-10,14.13,13.11,12.80,14.13,64103,-4.59
3,AT0000609607,2022-03-10,11.00,11.00,11.00,11.00,0,-1.79
4,AT0000644505,2022-03-10,96.50,89.90,88.10,96.60,794,-3.64
...,...,...,...,...,...,...,...,...
3219,XS2427363036,2022-03-10,20.17,20.17,20.17,20.17,0,1.35
3220,XS2427363895,2022-03-10,17.27,17.27,17.27,17.27,0,-5.49
3221,XS2427474023,2022-03-10,23.19,23.24,23.19,23.24,130,2.41
3222,XS2434891219,2022-03-10,3.55,3.55,3.54,3.59,101700,-7.33
